In [ ]:
import os
import pandas as pd

# 文件夹路径
tif_folder = "Scaled_1_100"  # 存放 Feature score 栅格
shap_mapping_file = "Expansion_FeatureScore_SHAP_mapping_with_LULC.xlsx"

# 获取 tif 文件名
tif_files = [f for f in os.listdir(tif_folder) if f.endswith(".tif")]
print("All tif files in folder:")
for f in tif_files:
    print(f)

# 读取映射表
df_map = pd.read_excel(shap_mapping_file)
df_map.columns = df_map.columns.str.strip()  # 去掉列名空格

# 第一列名为 Feature，取前 13 个名称（或所有）
feature_names = df_map["Feature"].unique()
print("\nAll Feature names in mapping table (from 'Feature' column):")
for f in feature_names:
    print(f)


In [ ]:
import rasterio
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# 文件夹路径
score_raster_folder = "Scaled_1_100"  # Feature score 栅格
shap_mapping_file = "Expansion_FeatureScore_SHAP_mapping_with_LULC.xlsx"
out_folder = "SHAP_rasters_Expansion"
os.makedirs(out_folder, exist_ok=True)

# 读取映射表
df_map = pd.read_excel(shap_mapping_file)
df_map.columns = df_map.columns.str.strip()
feature_names = df_map["Feature"].unique()

# 获取 tif 文件名
tif_files = [f for f in os.listdir(score_raster_folder) if f.endswith(".tif")]

# 建立 Feature 名称与 tif 文件的对应关系
mapping = {}
for f in feature_names:
    key = f.replace("Feature_", "").replace("_score","").lower()
    found = [t for t in tif_files if key in t.lower()]
    if not found:
        print(f"[Warning] No tif file matched for {f}")
    else:
        mapping[f] = found[0]  # 假设只匹配一个文件

print("Feature -> tif mapping:")
for k,v in mapping.items():
    print(f"{k} -> {v}")

# 生成 SHAP 栅格
for f_feat, tif_file in tqdm(mapping.items(), desc="Generating SHAP rasters"):
    # 提取 Feature 的映射表
    df_f = df_map[df_map["Feature"]==f_feat].set_index("Score")["SHAP_mean"].to_dict()
    
    raster_path = os.path.join(score_raster_folder, tif_file)
    with rasterio.open(raster_path) as src:
        arr = src.read(1)
        profile = src.profile
    
    # 创建 SHAP 栅格
    shp_arr = np.full_like(arr, np.nan, dtype=np.float32)
    valid_mask = arr > 0
    valid_scores = arr[valid_mask].astype(int)
    shp_arr[valid_mask] = np.array([df_f.get(s, np.nan) for s in valid_scores])
    
    # 输出
    outpath = os.path.join(out_folder, f"{tif_file.replace('.tif','')}_SHAP.tif")
    profile.update(dtype=rasterio.float32, nodata=np.nan)
    with rasterio.open(outpath, "w", **profile) as dst:
        dst.write(shp_arr, 1)

print("All Expansion SHAP rasters generated successfully.")


In [ ]:
import rasterio
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# 文件夹路径
score_raster_folder = "Scaled_1_100"  # Feature score 栅格
shap_mapping_file = "Shrink_FeatureScore_SHAP_mapping_with_LULC.xlsx"
out_folder = "SHAP_rasters_Shrink"
os.makedirs(out_folder, exist_ok=True)

# 读取映射表
df_map = pd.read_excel(shap_mapping_file)
df_map.columns = df_map.columns.str.strip()
feature_names = df_map["Feature"].unique()

# 获取 tif 文件名
tif_files = [f for f in os.listdir(score_raster_folder) if f.endswith(".tif")]

# 建立 Feature 名称与 tif 文件的对应关系
mapping = {}
for f in feature_names:
    key = f.replace("Feature_", "").replace("_score","").lower()
    found = [t for t in tif_files if key in t.lower()]
    if not found:
        print(f"[Warning] No tif file matched for {f}")
    else:
        mapping[f] = found[0]  # 假设只匹配一个文件

print("Feature -> tif mapping:")
for k,v in mapping.items():
    print(f"{k} -> {v}")

# 生成 SHAP 栅格
for f_feat, tif_file in tqdm(mapping.items(), desc="Generating SHRINK SHAP rasters"):
    # 提取 Feature 的映射表
    df_f = df_map[df_map["Feature"]==f_feat].set_index("Score")["SHAP_mean"].to_dict()
    
    raster_path = os.path.join(score_raster_folder, tif_file)
    with rasterio.open(raster_path) as src:
        arr = src.read(1)
        profile = src.profile
    
    # 创建 SHAP 栅格
    shp_arr = np.full_like(arr, np.nan, dtype=np.float32)
    valid_mask = arr > 0
    valid_scores = arr[valid_mask].astype(int)
    shp_arr[valid_mask] = np.array([df_f.get(s, np.nan) for s in valid_scores])
    
    # 输出
    outpath = os.path.join(out_folder, f"{tif_file.replace('.tif','')}_SHAP.tif")
    profile.update(dtype=rasterio.float32, nodata=np.nan)
    with rasterio.open(outpath, "w", **profile) as dst:
        dst.write(shp_arr, 1)

print("All Shrink SHAP rasters generated successfully.")
